# 1. IMPORT LIBRARY & SETUP CONNECTION

In [79]:
from datetime import datetime, timedelta
import pandas as pd
import pyodbc
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [80]:
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-MV8RPS5;"
    "DATABASE=Data_Python;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)

# 2. Import Data Frame From SQL SERVER

In [81]:
df_online_retail = pd.read_sql_query(''' 
SELECT *
  FROM [Data_Python].[dbo].[online_retail_II]
    ; ''', conn)
df_online_retail

C:\Users\Tran Minh Huy\AppData\Local\Temp\ipykernel_23844\1932557007.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_online_retail = pd.read_sql_query('''


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


# 3. CLEARNING DATA PROCESS

# 3.1 Standardization & Handle Missing Value

In [82]:
df_online_retail.columns = df_online_retail.columns.str.replace(' ', '_').str.lower()
print("0. Đã chuẩn hóa tên cột (snake_case).")
print(df_online_retail.columns)

# 3.1 Standardization & Handle Missing Value
# 1. Tính toán cột Sales và chuyển đổi kiểu dữ liệu
df_online_retail['sales'] = df_online_retail['quantity'] * df_online_retail['price']
df_online_retail['invoicedate'] = pd.to_datetime(df_online_retail['invoicedate'], errors='coerce') # Chuyển đổi an toàn
print("1. Calculated Sales and converted InvoiceDate to datetime.")

# 2. Xử lý Missing Critical Values (Bắt buộc phải xóa NaN Customer ID cho RFM)
initial_rows = df_online_retail.shape[0]
df_online_retail.dropna(subset=['customer_id'], inplace=True) # Dùng tên cột đã sửa: 'customer_id'
df_online_retail['customer_id'] = df_online_retail['customer_id'].astype(int)

df_online_retail['description'].fillna('UNKNOWN DESCRIPTION', inplace=True)
print(f"2. Dropped {initial_rows - df_online_retail.shape[0]} rows missing Customer ID (Bắt buộc).")

# 3. Remove Invalid Transactions & Duplicates
rows_before_invalid_check = df_online_retail.shape[0]

# Loại bỏ giao dịch bị hủy (Quantity <= 0)
df_online_retail = df_online_retail[df_online_retail['quantity'] > 0] 

# Loại bỏ giá trị không hợp lệ (Price <= 0)
df_online_retail = df_online_retail[df_online_retail['price'] > 0] 

# Loại bỏ hàng trùng lặp
df_online_retail.drop_duplicates(inplace=True)

lost_rows = rows_before_invalid_check - df_online_retail.shape[0]
print(f"3. Dropped {lost_rows} rows (Giao dịch hủy, giá 0 và Trùng lặp).")

# 4. Final Check (Sau khi làm sạch)
print("\n--- Cleaning Results (After all steps) ---")
print(f"Kích thước DataFrame sau khi làm sạch: {df_online_retail.shape}")
print(df_online_retail.info())

0. Đã chuẩn hóa tên cột (snake_case).
Index(['invoice', 'stockcode', 'description', 'quantity', 'invoicedate',
       'price', 'customer_id', 'country'],
      dtype='object')
1. Calculated Sales and converted InvoiceDate to datetime.


C:\Users\Tran Minh Huy\AppData\Local\Temp\ipykernel_23844\893760816.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_online_retail['description'].fillna('UNKNOWN DESCRIPTION', inplace=True)


2. Dropped 243007 rows missing Customer ID (Bắt buộc).
3. Dropped 44939 rows (Giao dịch hủy, giá 0 và Trùng lặp).

--- Cleaning Results (After all steps) ---
Kích thước DataFrame sau khi làm sạch: (779425, 9)
<class 'pandas.core.frame.DataFrame'>
Index: 779425 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoice      779425 non-null  object        
 1   stockcode    779425 non-null  object        
 2   description  779425 non-null  object        
 3   quantity     779425 non-null  int64         
 4   invoicedate  779425 non-null  datetime64[ns]
 5   price        779425 non-null  float64       
 6   customer_id  779425 non-null  int64         
 7   country      779425 non-null  object        
 8   sales        779425 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 59.5+ MB
None


In [83]:
df_online_retail

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,sales
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.399998
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.000000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.000000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.799995
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.000000
...,...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.599999
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.600000
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.600000
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680,France,14.849999


# 3.2 Standardize capitalization

In [84]:
# 3.2 Standardize capitalization (Renaming columns to snake_case)
column_mapping = {
    'Invoice': 'invoice',
    'StockCode': 'stock_code',
    'Description': 'description',
    'Quantity': 'quantity',
    'InvoiceDate': 'invoice_date',
    'Price': 'price',
    'Customer_ID': 'customer_id', 
    'Country': 'country',
    'Sales': 'sales' # Cột đã tính toán
}

# Apply rename
df_online_retail.rename(columns=column_mapping, inplace=True)
print(df_online_retail.columns)

Index(['invoice', 'stockcode', 'description', 'quantity', 'invoicedate',
       'price', 'customer_id', 'country', 'sales'],
      dtype='object')


# 3.3 Convert data type & 3.4 Detect & Remove Outliers Using IQR

In [85]:
Q1 = df_online_retail['sales'].quantile(0.25)
Q3 = df_online_retail['sales'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 3 * IQR 

rows_before_sales_outlier = df_online_retail.shape[0]
df_online_retail = df_online_retail[df_online_retail['sales'] <= upper_bound]
print(df_online_retail.info())

<class 'pandas.core.frame.DataFrame'>
Index: 738291 entries, 4 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoice      738291 non-null  object        
 1   stockcode    738291 non-null  object        
 2   description  738291 non-null  object        
 3   quantity     738291 non-null  int64         
 4   invoicedate  738291 non-null  datetime64[ns]
 5   price        738291 non-null  float64       
 6   customer_id  738291 non-null  int64         
 7   country      738291 non-null  object        
 8   sales        738291 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 56.3+ MB
None


# 4. FEATURE ENGINEERING RFM & MIN MAX SCALERS & VIP LABELING

In [86]:
df_rfm_input = df_online_retail.copy() 

# 1. Tính Reference Date (SỬ DỤNG TÊN CỘT ĐÚNG: 'invoicedate')
reference_date = df_rfm_input["invoicedate"].max() + pd.Timedelta(days=1)

# Groupby theo customer_id
rfm = df_rfm_input.groupby("customer_id").agg({
    # Q4: Recency (SỬ DỤNG TÊN CỘT ĐÚNG: "invoicedate")
    "invoicedate": lambda x: (reference_date - x.max()).days,
    
    # Q5: Frequency (Đếm số lượng invoice DUY NHẤT)
    "invoice": "nunique",
    
    # Q6: Monetary (tổng sales)
    "sales": "sum"
})

rfm.columns = ["R", "F", "M"]
print("\n>>> 4.1 Xây dựng RFM hoàn tất.")
print(rfm.head())


>>> 4.1 Xây dựng RFM hoàn tất.
               R   F            M
customer_id                      
12346        529  11   372.859995
12347          2   8  4128.510003
12348         75   5  1025.399995
12349         19   3  3270.689979
12350        310   1   334.399997


# Min-Max Scaler

In [87]:
scaler = MinMaxScaler()
rfm_scaled = pd.DataFrame(
    scaler.fit_transform(rfm),
    columns=["R_scaled", "F_scaled", "M_scaled"],
    index=rfm.index
)

# Kết hợp original và scaled
rfm_final = pd.concat([rfm, rfm_scaled], axis=1)

# Calculate Weighted RFM Score

In [88]:
rfm_final['RFM_SCORE'] = (1 - rfm_final['R_scaled']) * 0.3 + rfm_final['F_scaled'] * 0.5 + rfm_final['M_scaled'] * 0.2

rfm_final['VIP_Label'] = rfm_final['RFM_SCORE'].apply(lambda x: 1 if x > 0.75 else 0)
rfm_final['Label_RFM_SCORE'] = pd.cut(
    rfm_final['RFM_SCORE'],
    bins=[-1, 0.4, 0.7, 1.1],
    labels=["DELUX", "STANDARD", "VIP"]
)
print(">>> Tính RFM Score và Gán Nhãn VIP hoàn tất.")

>>> Tính RFM Score và Gán Nhãn VIP hoàn tất.


In [89]:
df_online_retail

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,sales
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.000000
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085,United Kingdom,39.599999
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.000000
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085,United Kingdom,59.499998
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085,United Kingdom,30.599999
...,...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.599999
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.600000
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.600000
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680,France,14.849999


# 5. TRAIN KNN MODEL & PREDICT NEW USER

In [90]:
X = rfm_final[['R_scaled', 'F_scaled', 'M_scaled']] # Features
y = rfm_final['VIP_Label'] # Label (Nhãn VIP)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X, y)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [91]:
new_users_data = {
    'R_scaled': [0.92, 0.88, 0.40, 0.30, 0.95, 0.50, 0.25, 0.70, 0.35, 0.60],
    'F_scaled': [0.10, 0.20, 0.60, 0.80, 0.05, 0.50, 0.90, 0.30, 0.75, 0.40],
    'M_scaled': [0.15, 0.30, 0.70, 0.80, 0.10, 0.55, 0.85, 0.40, 0.75, 0.45]
}
df_new = pd.DataFrame(new_users_data)

new_predictions = knn.predict(df_new)
df_new['Customer_ID'] = [f'C{i:03d}' for i in range(1, 11)]
df_new['Predicted_Segment'] = new_predictions

In [92]:
print("\n=== 2. KIỂM TRA KẾT QUẢ DỰ ĐOÁN KNN (df_new) ===")
# Hiển thị khách hàng mới và nhãn dự đoán của chúng
print(df_new[['Customer_ID', 'R_scaled', 'F_scaled', 'M_scaled', 'Predicted_Segment']])


=== 2. KIỂM TRA KẾT QUẢ DỰ ĐOÁN KNN (df_new) ===
  Customer_ID  R_scaled  F_scaled  M_scaled  Predicted_Segment
0        C001      0.92      0.10      0.15                  0
1        C002      0.88      0.20      0.30                  0
2        C003      0.40      0.60      0.70                  0
3        C004      0.30      0.80      0.80                  0
4        C005      0.95      0.05      0.10                  0
5        C006      0.50      0.50      0.55                  0
6        C007      0.25      0.90      0.85                  0
7        C008      0.70      0.30      0.40                  0
8        C009      0.35      0.75      0.75                  0
9        C010      0.60      0.40      0.45                  0


In [93]:
print("\n=== 1. KIỂM TRA RFM VÀ NHÃN VIP CỦA KHÁCH HÀNG CŨ (rfm_final) ===")
# Hiển thị các cột quan trọng nhất: R, F, M, các giá trị scaled, Score và Nhãn
print(rfm_final[['R', 'F', 'M', 'R_scaled', 'RFM_SCORE', 'VIP_Label', 'Label_RFM_SCORE']].head(10))


=== 1. KIỂM TRA RFM VÀ NHÃN VIP CỦA KHÁCH HÀNG CŨ (rfm_final) ===
               R   F            M  R_scaled  RFM_SCORE  VIP_Label  \
customer_id                                                         
12346        529  11   372.859995  0.715447   0.099122          0   
12347          2   8  4128.510003  0.001355   0.312881          0   
12348         75   5  1025.399995  0.100271   0.276249          0   
12349         19   3  3270.689979  0.024390   0.298457          0   
12350        310   1   334.399997  0.418699   0.174705          0   
12351        375   1   300.929998  0.506775   0.148250          0   
12352         36   9  1849.539987  0.047425   0.298244          0   
12353        204   2   406.759997  0.275068   0.219203          0   
12354        232   1  1079.400008  0.313008   0.207117          0   
12355        214   2   752.609991  0.288618   0.215465          0   

            Label_RFM_SCORE  
customer_id                  
12346                 DELUX  
12347         

In [94]:
# Kiểm tra phân phối Nhãn VIP
print("\n--- Phân phối Nhãn VIP (Historical Data) ---")
print(rfm_final['VIP_Label'].value_counts())

# Kiểm tra điểm số Min/Max
print(f"\nRFM Score Trung bình: {rfm_final['RFM_SCORE'].mean():.4f}")
print(f"RFM Score Cao nhất: {rfm_final['RFM_SCORE'].max():.4f}")
print(f"RFM Score Thấp nhất: {rfm_final['RFM_SCORE'].min():.4f}")


--- Phân phối Nhãn VIP (Historical Data) ---
VIP_Label
0    5721
1       2
Name: count, dtype: int64

RFM Score Trung bình: 0.2269
RFM Score Cao nhất: 1.0000
RFM Score Thấp nhất: 0.0001


In [ ]:
# 6. Save Clear Data Into Database To Visiualization ti

In [95]:
import pyodbc
df_to_save = rfm_final.reset_index()

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-MV8RPS5;"
    "DATABASE=Data_Python;"  
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)
cursor = conn.cursor()

# --- Map pandas dtype -> SQL Server type ---
dtype_map = lambda dt: "INT" if pd.api.types.is_integer_dtype(dt) else \
                       "FLOAT" if pd.api.types.is_float_dtype(dt) else \
                       "BIT" if pd.api.types.is_bool_dtype(dt) else \
                       "DATETIME" if pd.api.types.is_datetime64_any_dtype(dt) else \
                       "VARCHAR(255)" 

table_name = "RFM_Online_Retail_Results" 

# --- Drop + Create table ---
cursor.execute(f"IF OBJECT_ID('dbo.{table_name}', 'U') IS NOT NULL DROP TABLE dbo.{table_name}")
# Sử dụng df_to_save (rfm_final đã reset index)
cols = ", ".join([f"[{c}] {dtype_map(df_to_save[c].dtype)}" for c in df_to_save.columns])
cursor.execute(f"CREATE TABLE dbo.{table_name} ({cols})")

# --- Insert rows ---
cursor.fast_executemany = True
# Sử dụng df_to_save
cursor.executemany(f"INSERT INTO dbo.{table_name} VALUES ({','.join(['?']*len(df_to_save.columns))})", df_to_save.values.tolist())
conn.commit()
conn.close()
